In [ ]:
'''
환경 : 쥬피터 노트북
0. https://pypi.org/project/googletrans/
번역 api - 각 단계 진행 시 해당 string 번역하여 값 반환
하이퍼링크 제거 메서드 생성

1. 테이블 정의 - 레시피, 요리정보, 재료, 요리단계

레시피 - Recipe
1. id - db 자동배정
2. foodName : title
3. foodImg : image
4. tempId : spoonacular api recipeId

요리정보 - FoodInformation
1. id - db 자동 배정
2. text - summary
3. cookingTime - readyInMinutes
4. serving - servings

재료 - Ingredients 항목에서 가져오기
1. id - db 자동배정
2. ingredientName : ingredientName
3. amount : amount

요리 단계 - CookStep : analyzedInstruction 항목에서 가져오기
1. id - db 자동
2. stepNumber - number
3. text - step
4. stepImg - null로 통일

2. docker - mysql
docker volume 설정으로 레시피 및 db 저장 폴더 지정
파이썬 mysql 연결 flask x

3. 레시피 아이디 들고와서 레시피 객체 생성 후 db 저장
4. 레시피 아이디를 리스트로 받아오기
5. 받아온 아이디를 기반으로 단일 레시피 get 요청 진행
    5-1. 요리 정보 : 데이터 분리, body에서 summary 추출,이미지 링크 삭제, 그리고 매핑 및 객체 생성,
    5-2. 재료 : 테이블과 각 값 매핑해서 객체 생성,
    5-3. 요리 단계 : 테이블과 각 값 매핑해서 객체 생성
    각 순서마다 db에 상황마다 바로 전송
    
API key : 03351b92426b464499f9b05790d6894b
'''

In [ ]:
! pip install sqlalchemy
! pip install googletrans==3.1.0a0

In [ ]:
class Recipe:
    def __init__(self, recipe_id, food_name, food_img, temp_id):
        self.recipe_id = recipe_id  # DB에서 자동으로 할당됨
        self.food_name = food_name
        self.food_img = food_img
        self.temp_id = temp_id

    def updateFoodName(self, food_name):
        self.food_name = food_name


class FoodInformation:
    def __init__(self, recipe_id, text, cooking_time, serving):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.text = text
        self.cooking_time = cooking_time
        self.serving = serving

    def updateText(self, text):
        self.text = text


class Ingredient:
    def __init__(self, recipe_id, ingredient_name, amount):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.ingredient_name = ingredient_name
        self.amount = amount

    def updateIngredient(self, ingredient_name):
        self.ingredient_name = ingredient_name

class CookStep:
    def __init__(self, recipe_id, step_number, text, step_img):
        self.id = None  # DB에서 자동으로 할당됨
        self.recipe_id = recipe_id
        self.step_number = step_number
        self.text = text
        self.step_img = step_img

    def updateCookstep(self, text):
        self.text = text
       
       




In [ ]:
! pip install PyMySQL;

In [ ]:
from sqlalchemy.dialects.oracle import LONG
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime, create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects import mysql


# 데이터베이스 연결 설정
db_name = "tastyhub"
db_host = "localhost"
db_username = "root"
db_password = "1234"

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}")

# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()

# 기본 클래스 생성
Base = declarative_base()


# Recipe 클래스를 데이터베이스 테이블로 매핑
class TRecipe(Base):
    __tablename__ = 'recipes'

    recipe_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    food_name = Column(String(255))
    food_img = Column(String(255))
    # created_at = Column(DateTime)
    # modified_at = Column(DateTime)
    # user_id = Column(Integer,ForeignKey('users.user_id'))

    food_information = relationship('TFoodInformation', back_populates='recipe')
    ingredient = relationship('TIngredient', back_populates='recipe')
    cook_steps = relationship('TCookStep', back_populates='recipe')

# FoodInformation 클래스를 데이터베이스 테이블로 매핑
class TFoodInformation(Base):
    __tablename__ = 'food_information'

    food_information_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    text = Column(String(1000))
    cooking_time = Column(Integer)
    serving = Column(Integer)
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='food_information')

# Ingredient 클래스를 데이터베이스 테이블로 매핑
class TIngredient(Base):
    __tablename__ = 'ingredient'

    ingredient_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    ingredient_name = Column(String(255))
    amount = Column(String(100))
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='ingredient')

# CookStep 클래스를 데이터베이스 테이블로 매핑
class TCookStep(Base):
    __tablename__ = 'cook_steps'
    __tablename__ = 'cook_steps'

    cookstep_id = Column(mysql.BIGINT, primary_key=True, autoincrement=True)
    
    recipe_id = Column(mysql.BIGINT, ForeignKey('recipes.recipe_id'))
    step_number = Column(Integer)
    text = Column(String(1000))
    step_img_url = Column(String(255))
    created_at = Column(DateTime)
    modified_at = Column(DateTime)

    recipe = relationship('TRecipe', back_populates='cook_steps')
# 테이블 생성
Base.metadata.create_all(engine)

# 세션 종료
session.close()

In [ ]:
# Recipe 객체를 TRecipe 객체로 변환하는 함수
def convert_to_trecipe(recipe):
    print(recipe.food_name)
    trecipe = TRecipe(
        # recipe_id=recipe.recipe_id,
        food_name=recipe.food_name,
        food_img=recipe.food_img,
        # created_at=DateTime.
    )
    print(trecipe.food_name)
    return trecipe

# FoodInformation 객체를 TFoodInformation 객체로 변환하는 함수
def convert_to_tfoodinformation(food_information,trecipe):
    tfood_information = TFoodInformation(
        recipe_id=food_information.recipe_id,
        text=food_information.text,
        cooking_time=food_information.cooking_time,
        serving=food_information.serving,
        recipe = trecipe
    )
    return tfood_information

# Ingredient 객체를 TIngredient 객체로 변환하는 함수
def convert_to_tingredient(ingredient,trecipe):
    tingredient = TIngredient(
        recipe_id=ingredient.recipe_id,
        ingredient_name=ingredient.ingredient_name,
        amount=ingredient.amount,
        recipe = trecipe
    )
    return tingredient

# CookStep 객체를 TCookStep 객체로 변환하는 함수
def convert_to_tcookstep(cook_step,trecipe):
    tcook_step = TCookStep(
        recipe_id=cook_step.recipe_id,
        step_number=cook_step.step_number,
        text=cook_step.text,
        step_img_url=cook_step.step_img,
        recipe = trecipe
    )
    return tcook_step


In [ ]:
import requests

all_search_url = 'https://api.spoonacular.com/recipes/complexSearch?number=&apiKey=03351b92426b464499f9b05790d6894b'
response = requests.get(all_search_url)

In [ ]:
print(response.text)

In [ ]:
import json

In [ ]:
data = json.loads(response.text)
list_id = 0
recipes = []
# Iterate over each item in the list of results
for result in data["results"]:
    # Access each attribute of the result
    print(type(result))
    recipe_id = list_id
    title = result["title"]
    image = result["image"]
    temp_id = result["id"]
    
    recipe = Recipe(recipe_id, title, image, temp_id)
    print(recipe)

    recipes.append(recipe)
    list_id += 1

print(recipes[0].food_name)  # Just for formatting

print(len(recipes))

In [ ]:
foodInformations = []
ingredients = []
cookSteps = []
for obj in recipes:
    recipe_temp_id = obj.temp_id
    if obj.recipe_id == 3 :
        break
    recipe_search_url = 'https://api.spoonacular.com/recipes/informationBulk?ids=' + str(
        recipe_temp_id) + '&includeNutrition=false&apiKey=03351b92426b464499f9b05790d6894b'
    response = requests.get(recipe_search_url)
    text = response.text
    data = json.loads(text)
    for foodInfo in data:
        text = foodInfo['summary']
        cookingTime = foodInfo['readyInMinutes']
        serving = foodInfo['servings']
        foodInformation = FoodInformation(obj.recipe_id,text,cookingTime,serving)
        foodInformations.append(foodInformation)

        for ing in foodInfo['extendedIngredients']:
            ingredient_name = ing['name']
            amount = ing['amount']
            ingredient = Ingredient(obj.recipe_id,ingredient_name, amount)
            ingredients.append(ingredient)

        for cks in foodInfo['analyzedInstructions']:
            ck = cks['steps']
            for c in ck:
                stepNumber = c['number']
                text = c['step']
                stepImg = "afterRefactor"
                cookStep = CookStep(obj.recipe_id,stepNumber,text,stepImg)
                cookSteps.append(cookStep)
    


In [ ]:
for rec in recipes:
    id = rec.recipe_id
    print(id)
    if id == 3:
        break
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            print(fi.text)
            print(fi.cooking_time)
            print(fi.serving)
            print()

    for ig in ingredients:
        if id == ig.recipe_id:
            print(ig.ingredient_name)
            print(ig.amount)
            print()

    for cs in cookSteps:
        if id == cs.recipe_id:
            print(cs.text)
            print(cs.step_number)
            print(cs.step_img)
            print()

    print("---------다음 레시피--------")

In [ ]:
import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


In [ ]:
for fi in foodInformations:
    text = fi.text
    text = remove_html_tags(text)
    fi.updateText(text)
    print(fi.text)

In [ ]:
# 번역 메서드
from googletrans import Translator
#\ https://pypi.org/project/googletrans/

translator = Translator()

for rec in recipes:
    id = rec.recipe_id
    if id == 3:
        break
    text = rec.food_name
    translated = translator.translate(text,src="en", dest="ko")

    rec.updateFoodName(str(translated.text))
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            fi.updateText(str(translator.translate(fi.text,dest="ko").text))


    for ig in ingredients:
        if id == ig.recipe_id:
            ig.updateIngredient(str(translator.translate(ig.ingredient_name, dest="ko").text))


    for cs in cookSteps:
        if id == cs.recipe_id:
            cs.updateCookstep(str(translator.translate(cs.text, dest="ko").text))




In [ ]:
for rec in recipes:
    id = rec.recipe_id
    print(id)
    if id == 3:
        break
    print(rec.food_name)
    for fi in foodInformations:
        if id == fi.recipe_id:
            print(fi.text)
            print(fi.cooking_time)
            print(fi.serving)
            print()

    for ig in ingredients:
        if id == ig.recipe_id:
            print(ig.ingredient_name)
            print(ig.amount)
            print()

    for cs in cookSteps:
        if id == cs.recipe_id:
            print(cs.text)
            print(cs.step_number)
            print(cs.step_img)
            print()

    print("---------다음 레시피--------")


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

# 데이터베이스 연결 설정
db_name = "tastyhub"
db_host = "localhost"
db_username = "root"
db_password = "1234"

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}")

# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()



In [ ]:
Base = declarative_base()
try:
    print("start")
    for rec in recipes:
        trecipe = convert_to_trecipe(rec)
        session.add(trecipe)
        session.commit()
    
    recipe_objects = session.query(TRecipe).all()
    print(type(recipe_objects))
    
    for ro in recipe_objects:
        for fi in foodInformations:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            print(fi.recipe_id)
            print(ro.recipe_id)
            if fi.recipe_id+1==ro.recipe_id:
                tfoodinformation = convert_to_tfoodinformation(fi,ro)
                print(tfoodinformation.recipe_id)
                session.add(tfoodinformation)

        for ing in ingredients:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            if ing.recipe_id+1 == ro.recipe_id:
                tingredient = convert_to_tingredient(ing,ro)
                session.add(tingredient)

        for cs in cookSteps:  # your_objects에는 객체들의 리스트가 들어가야 합니다.
            if cs.recipe_id+1 == ro.recipe_id:
                tcookstep = convert_to_tcookstep(cs,ro)
                session.add(tcookstep)
            
    session.commit()



except Exception as e:
    session.rollback()
    print("Error:", e)

# 세션 종료
session.close()